In [1]:
import numpy as np
import pandas as pd
import csv as csv
from sklearn.ensemble import RandomForestClassifier

In [2]:
# reads in train and test data sets

train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

In [3]:
# remaps strings to ints

train_set['Sex'] = train_set['Sex'].map({'female': 0, 'male': 1}).astype(int)
train_set['Embarked'][train_set['Embarked'].isnull()] = 'S'
train_set['Embarked'] = train_set['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
test_set['Sex'] = test_set['Sex'].map({'female': 0, 'male': 1}).astype(int)
test_set['Embarked'][test_set['Embarked'].isnull()] = 'S'
test_set['Embarked'] = test_set['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

//anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
# fills missing Age values

median_ages = np.zeros((2,3))
for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = train_set[(train_set['Sex'] == i) & \
                              (train_set['Pclass'] == j+1)]['Age'].dropna().median()
train_set['AgeFill'] = train_set['Age']
for i in range(0, 2):
    for j in range(0, 3):
        train_set.loc[ (train_set.Age.isnull()) & (train_set.Sex == i) & (train_set.Pclass == j+1),\
                'AgeFill'] = median_ages[i,j]
train_set['FamilySize'] = train_set['SibSp'] + train_set['Parch']

median_ages = np.zeros((2,3))
for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = test_set[(test_set['Sex'] == i) & \
                              (test_set['Pclass'] == j+1)]['Age'].dropna().median()
test_set['AgeFill'] = test_set['Age']
for i in range(0, 2):
    for j in range(0, 3):
        test_set.loc[ (test_set.Age.isnull()) & (test_set.Sex == i) & (test_set.Pclass == j+1),\
                'AgeFill'] = median_ages[i,j]
        
# creates Family Size feature
test_set['FamilySize'] = test_set['SibSp'] + test_set['Parch']

In [5]:
# sets Fare == 0 to mean of Pclass Fares

mean_fare = np.zeros((3))
for i in range(0, 3):
    mean_fare[i] = train_set[(train_set['Pclass'] == i + 1)]['Fare'].dropna().mean()
for i in range(0, 3):
    train_set.loc[(train_set.Fare == 0) & (train_set.Pclass == i + 1), 'Fare'] = mean_fare[i]
for i in range(0, 3):
    mean_fare[i] = test_set[(test_set['Pclass'] == i + 1)]['Fare'].dropna().mean()
for i in range(0, 3):
    test_set.loc[(test_set.Fare == 0) & (test_set.Pclass == i + 1), 'Fare'] = mean_fare[i]

In [6]:
# sets missing Fare values

for i in range(0, 3):
    train_set.loc[(train_set.Fare.isnull()), 'Fare'] = mean_fare[i]
for i in range(0, 3):
    test_set.loc[(test_set.Fare.isnull()), 'Fare'] = mean_fare[i]

In [7]:
# converts Cabin to Deck as int

train_set['Deck'] = train_set['Cabin']
train_set['Deck'][train_set['Deck'].isnull()] = 'G'
for i in range(0,len(train_set['Deck'])):
    train_set['Deck'][i] = ord(train_set['Deck'][i][0]) - ord('A')

test_set['Deck'] = test_set['Cabin']
test_set['Deck'][test_set['Deck'].isnull()] = 'G'
for i in range(0,len(test_set['Deck'])):
    test_set['Deck'][i] = ord(test_set['Deck'][i][0]) - ord('A')

//anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [8]:
# Sets missing value Deck to mean of Pclass Deck.

mean_deck = np.zeros((3))
for i in range(0, 3):
    mean_deck[i] = train_set[(train_set['Pclass'] == i + 1) & (train_set['Deck'] != 6)]['Deck'].dropna().mean()
mean_deck = mean_deck.round()
for i in range(0, 3):
    train_set.loc[(train_set.Deck == 6) & (train_set.Pclass == i + 1), 'Deck'] = mean_deck[i]
for i in range(0, 3):
    mean_deck[i] = test_set[(test_set['Pclass'] == i + 1) & (test_set['Deck'] != 6)]['Deck'].dropna().mean()
mean_deck = mean_deck.round()
for i in range(0, 3):
    test_set.loc[(test_set.Deck == 6) & (test_set.Pclass == i + 1), 'Deck'] = mean_deck[i]

In [9]:
# pulls Title from Name

train_set['Title'] = train_set['Name']
rows = len(train_set)
for i in range(0, rows):
    start = train_set['Name'][i].find(', ') + 2
    end = train_set['Name'][i].find('.')
    train_set.loc[[i], 'Title'] = train_set['Name'][i][start:end]

test_set['Title'] = test_set['Name']
rows = len(test_set)
for i in range(0, rows):
    start = test_set['Name'][i].find(', ') + 2
    end = test_set['Name'][i].find('.')
    test_set.loc[[i], 'Title'] = test_set['Name'][i][start:end]
    
train_set['Title'] = train_set['Title'].map({'Mr':0, 'Miss':1, 'Mrs':2, 'Master':3, 'Dr':4, 
                                             'Rev':5, 'Major':6, 'Mlle':1, 'Col':6, 
                                             'Jonkheer':3, 'Sir': 6, 'Mme': 2, 
                                             'the Countess': 7, 'Don': 6, 'Ms': 1, 
                                             'Lady': 7, 'Capt': 6, 'Dona': 7}).astype(int)
test_set['Title'] = test_set['Title'].map({'Mr':0, 'Miss':1, 'Mrs':2, 'Master':3, 'Dr':4, 
                                             'Rev':5, 'Major':6, 'Mlle':1, 'Col':6, 
                                             'Jonkheer':3, 'Sir': 6, 'Mme': 2, 
                                             'the Countess': 7, 'Don': 6, 'Ms': 1, 
                                             'Lady': 7, 'Capt': 6, 'Dona': 7}).astype(int)

In [10]:
train_set.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeFill,FamilySize,Title
count,891.000000,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.647587,29.699118,0.523008,0.381594,32.876990,0.361392,29.112424,0.904602,0.756453
std,257.353842,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057,49.690114,0.635673,13.304424,1.613459,1.141727
min,1.000000,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,4.012500,0.000000,0.420000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000,7.925000,0.000000,21.500000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,1.000000,28.000000,0.000000,0.000000,14.500000,0.000000,26.000000,0.000000,0.000000
75%,668.500000,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.275000,1.000000,36.000000,1.000000,1.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000,80.000000,10.000000,7.000000
